In [3]:
'''
packages
'''
import sys
import keras
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import random
import pickle

Using TensorFlow backend.


In [4]:
#from keras_metrics import KerasMetrics

In [9]:
import coding_function as cf
import decoding_function as df
import csv
from Files import nodes

In [10]:
with open('Files/training_data_set.csv','r') as csvfile:
    readcsv = csv.reader(csvfile, delimiter = ',')
    phoneticinf = []
    phoneticI = []
    for row in readcsv:
        phoneticinf.append(row[1])
        phoneticI.append(row[3])

In [11]:
cf.dataTest(phoneticinf,phoneticI) # careful with duplicates on your dataset

'done'

In [12]:
# for key in dictioinf.keys(): 
#      if key in phoneticinf:
#         phoneticinf.remove(key)
# phoneticinf

In [13]:
dictioinf = {}
dictioI = {}

for item in phoneticinf:
    dictioinf[item] = cf.coding(item)

for item in phoneticI:
    dictioI[item] = cf.coding(item)

In [14]:
len(phoneticinf),len(phoneticI),len(dictioinf),len(dictioI)

(410, 410, 410, 410)

In [15]:
X = dictioinf.values() #input column
print (len(X))
Y = dictioI.values() #output column

410


In [16]:
X
np.array(X)
print(len(X),len(Y))

410 410


In [17]:
X = np.array(list(X))
Y = np.array(list(Y))

In [18]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 1, 0]])

In [19]:
'''
2. Define model
'''

model = Sequential()
model.add(Dense(460, input_shape=(460,), activation='sigmoid'))


In [20]:
'''
3. Evaluate model
'''



'\n3. Evaluate model\n'

In [21]:
'''
4. Compile model
'''

model.compile(loss='mean_squared_error', optimizer='adadelta')

In [24]:
'''
4. Fit model
'''
model.fit(X,Y, epochs=200, batch_size=20)

Epoch 1/200
410/410 [==============================] - 0s - loss: 0.0403     
Epoch 2/200
410/410 [==============================] - 0s - loss: 0.0403     
Epoch 3/200
410/410 [==============================] - 0s - loss: 0.0402     
Epoch 4/200
410/410 [==============================] - 0s - loss: 0.0401     
Epoch 5/200
410/410 [=================

In [25]:
gostar = "#gosta#"
Gostar = cf.coding(gostar)
tomar = "#toma#"
Tomar = cf.coding(tomar)
mexer = "mexe"
Mexer = cf.coding(mexer)
Test_list = [Gostar,Tomar,Mexer]


In [26]:
Test_array = np.array(Test_list)
Test_array.shape

(3, 460)

In [27]:
prediction = model.predict(Test_array)
#print(prediction)

In [28]:
type(prediction)

numpy.ndarray

In [32]:
print(list(prediction[0]))

[0.019040588, 0.012052633, 0.35236615, 0.10005063, 0.32916832, 0.073083811, 0.097307645, 0.31947699, 0.27201638, 0.11171351, 0.040509794, 0.022570252, 0.83192605, 0.21355128, 0.43425792, 0.2317199, 0.1625178, 0.53424931, 0.56238937, 0.29616728, 0.035984173, 0.12696837, 0.18102938, 0.17621739, 0.1549129, 0.10456822, 0.13573548, 0.16392741, 0.19988234, 0.086747974, 0.021539299, 0.018603889, 0.10955386, 0.039493751, 0.10809998, 0.045770492, 0.068783499, 0.14489792, 0.066018559, 0.09429688, 0.094017774, 0.026666235, 0.069293737, 0.02895497, 0.14700267, 0.10910633, 0.066770062, 0.091047622, 0.11161723, 0.041579932, 0.86798382, 0.071181506, 0.11288071, 0.10100957, 0.84827143, 0.094324648, 0.71517462, 0.34046543, 0.8886947, 0.10013086, 0.031530045, 0.9209848, 0.02345353, 0.082169451, 0.87085783, 0.044378985, 0.90921474, 0.018885203, 0.86772192, 0.12760341, 0.08795578, 0.052530669, 0.033222917, 0.14583109, 0.06420096, 0.085360304, 0.10217992, 0.033955492, 0.056096494, 0.088660903, 0.28317007, 

In [493]:
thresholded = np.where(prediction >= 0.600, 1, 0)

In [1]:
prediction

NameError: name 'prediction' is not defined

In [494]:
lists = thresholded.tolist()

In [507]:
algo=lists[1]
algo.count(1)

51

In [508]:
algo

[0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [509]:
wickelfeatures_list = nodes.nds

In [510]:
resposta = cf.vector2string(algo,wickelfeatures_list)

In [511]:
df.decoding(resposta)

'#to'

In [457]:
df.decoding(cf.vector2string(cf.coding("#gOstu#"),wickelfeatures_list))

'#gOstu#'

In [499]:
cf.coding("#gosta#")

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
